In [3]:
import pandas as pd


df = pd.read_csv("fifa21.csv")


df.info()


df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 77 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                18979 non-null  int64 
 1   Name              18979 non-null  object
 2   LongName          18979 non-null  object
 3   photoUrl          18979 non-null  object
 4   playerUrl         18979 non-null  object
 5   Nationality       18979 non-null  object
 6   Age               18979 non-null  int64 
 7   ↓OVA              18979 non-null  int64 
 8   POT               18979 non-null  int64 
 9   Club              18979 non-null  object
 10  Contract          18979 non-null  object
 11  Positions         18979 non-null  object
 12  Height            18979 non-null  object
 13  Weight            18979 non-null  object
 14  Preferred Foot    18979 non-null  object
 15  BOV               18979 non-null  int64 
 16  Best Position     18979 non-null  object
 17  Joined      

C:\Users\Moazam\AppData\Local\Temp\ipykernel_2264\3253979602.py:4: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("fifa21.csv")


,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,158023,L. Messi,Lionel Messi,https://cdn.sofifa.com/players/158/023/21_60.png,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,33,93,93,\n\n\n\nFC Barcelona,...,Medium,Low,5 ★,85,92,91,95,38,65,771
1,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,https://cdn.sofifa.com/players/020/801/21_60.png,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,35,92,92,\n\n\n\nJuventus,...,High,Low,5 ★,89,93,81,89,35,77,562
2,200389,J. Oblak,Jan Oblak,https://cdn.sofifa.com/players/200/389/21_60.png,http://sofifa.com/player/200389/jan-oblak/210006/,Slovenia,27,91,93,\n\n\n\nAtlético Madrid,...,Medium,Medium,3 ★,87,92,78,90,52,90,150
3,192985,K. De Bruyne,Kevin De Bruyne,https://cdn.sofifa.com/players/192/985/21_60.png,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,29,91,91,\n\n\n\nManchester City,...,High,High,4 ★,76,86,93,88,64,78,207
4,190871,Neymar Jr,Neymar da Silva Santos Jr.,https://cdn.sofifa.com/players/190/871/21_60.png,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,28,91,91,\n\n\n\nParis Saint-Germain,...,High,Medium,5 ★,91,85,86,94,36,59,595


In [7]:
# Missing values
missing_values = df.isnull().sum().sort_values(ascending=False)

# Data types
data_types = df.dtypes


sample_columns = df[['Height', 'Weight', 'Wage', 'Value', 'Release Clause']].sample(5)


duplicates = df.duplicated().sum()


In [15]:
import numpy as np

# Handle missing values in numerical columns by filling with the median
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].apply(lambda x: x.fillna(x.median()))

# Handle missing values in categorical columns by filling with 'Unknown'
categorical_cols = df.select_dtypes(include=['object']).columns
df[categorical_cols] = df[categorical_cols].fillna('Unknown')


In [17]:
import re
import pandas as pd


def convert_height(val):
    try:
        val = str(val).strip().replace('"', '')
        if "'" in val:
            feet, inches = val.split("'")
            return round(int(feet) * 30.48 + int(inches) * 2.54)
        elif "cm" in val:
            return int(re.findall(r'\d+', val)[0])
        elif val.isdigit():
            return int(val)
    except:
        return np.nan

df['Height'] = df['Height'].apply(lambda x: convert_height(x))

# --- Convert weight like "165lbs" to integer (optional cleanup if needed) ---
if 'Weight' in df.columns:
    df['Weight'] = df['Weight'].astype(str).str.replace('lbs', '', regex=False).str.strip()
    df['Weight'] = pd.to_numeric(df['Weight'], errors='coerce')


def convert_money(val):
    val = str(val).replace('€', '').strip()
    try:
        if 'M' in val:
            return int(float(val.replace('M', '')) * 1_000_000)
        elif 'K' in val:
            return int(float(val.replace('K', '')) * 1_000)
        return int(float(val))
    except:
        return np.nan

for col in ['Wage', 'Value', 'Release Clause']:
    if col in df.columns:
        df[col] = df[col].apply(convert_money)

if 'Contract Valid Until' in df.columns:
    df['Contract Valid Until'] = pd.to_datetime(df['Contract Valid Until'], errors='coerce')
    df['Contract Valid Until'] = df['Contract Valid Until'].dt.strftime('%Y-%m-%d')


In [19]:
# --- Convert numerical columns to appropriate numeric types ---
numerical_columns = ['Age', 'Height', 'Weight', 'Wage', 'Value', 'Release Clause']
for col in numerical_columns:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# --- Convert categorical columns to string type ---
categorical_columns = ['Position', 'Nationality', 'Club']
for col in categorical_columns:
    if col in df.columns:
        df[col] = df[col].astype(str)


In [21]:

columns_to_drop = ['LongName', 'photoUrl', 'playerUrl', 'ID']  # Add any columns you consider unnecessary
df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)


df['Club'] = df['Club'].str.replace(r'(?i)^FC\s+', '', regex=True).str.strip()


club_replacements = {
    'Manchester United FC': 'Manchester United',
    'FC Barcelona': 'Barcelona',
    'Real Madrid CF': 'Real Madrid',
    'FC Bayern München': 'Bayern Munich'
}
df['Club'] = df['Club'].replace(club_replacements)


In [25]:
# --- Create Player Role Column based on Position ---
def get_player_role(position):
    # Example mapping: you can adjust it based on your dataset's positions
    forwards = ['ST', 'CF', 'RW', 'LW']
    midfielders = ['CM', 'CDM', 'CAM', 'RM', 'LM']
    defenders = ['CB', 'RB', 'LB', 'RWB', 'LWB']
    goalkeepers = ['GK']

    if position in forwards:
        return 'Forward'
    elif position in midfielders:
        return 'Midfielder'
    elif position in defenders:
        return 'Defender'
    elif position in goalkeepers:
        return 'Goalkeeper'
    return 'Unknown'  # In case of an unrecognized position


df['Player Role'] = df['Best Position'].apply(get_player_role)


df.to_csv('cleaned_fifa21.csv', index=False)

# Verify the first few rows of the updated DataFrame
print(df.head())


                Name Nationality  Age  ↓OVA  POT                 Club  \
0           L. Messi   Argentina   33    93   93            Barcelona   
1  Cristiano Ronaldo    Portugal   35    92   92             Juventus   
2           J. Oblak    Slovenia   27    91   93      Atlético Madrid   
3       K. De Bruyne     Belgium   29    91   91      Manchester City   
4          Neymar Jr      Brazil   28    91   91  Paris Saint-Germain   

      Contract   Positions  Height  Weight  ...     D/W   IR PAC SHO PAS  DRI  \
0  2004 ~ 2021  RW, ST, CF     170   171.0  ...     Low  5 ★  85  92  91   95   
1  2018 ~ 2022      ST, LW     187   171.0  ...     Low  5 ★  89  93  81   89   
2  2014 ~ 2023          GK     188   171.0  ...  Medium  3 ★  87  92  78   90   
3  2015 ~ 2023     CAM, CM     181   171.0  ...    High  4 ★  76  86  93   88   
4  2017 ~ 2022     LW, CAM     175   171.0  ...  Medium  5 ★  91  85  86   94   

   DEF  PHY  Hits  Player Role  
0   38   65   771      Forward  
1   35  

In [31]:
pip install mysql-connector-python sqlalchemy pandas


   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
    --------------------------------------- 0.3/16.4 MB ? eta -:--:--
   -- ------------------------------------- 1.0/16.4 MB 3.1 MB/s eta 0:00:05
   --- ------------------------------------ 1.3/16.4 MB 2.5 MB/s eta 0:00:07
   ---- ----------------------------------- 1.8/16.4 MB 3.2 MB/s eta 0:00:05
   ------ --------------------------------- 2.6/16.4 MB 2.8 MB/s eta 0:00:05
   ------- -------------------------------- 3.1/16.4 MB 2.8 MB/s eta 0:00:05
   -------- ------------------------------- 3.4/16.4 MB 2.8 MB/s eta 0:00:05
   -------- ------------------------------- 3.7/16.4 MB 2.4 MB/s eta 0:00:06
   --------- ------------------------------ 3.9/16.4 MB 2.3 MB/s eta 0:00:06
   ------------ --------------------------- 5.2/16.4 MB 2.6 MB/s eta 0:00:05
   ------------ --------------------------- 5.2/16.4 MB 2.6 MB/s eta 0:00:05
   ------------- ---

In [64]:
import pandas as pd


df = pd.read_csv('cleaned_fifa21.csv')


df = df.rename(columns={
    'Name': 'player_name',
    'Nationality': 'nationality',
    'Club': 'club_name',
    'Best Position': 'player_position',
    'Age': 'age',
    'Height': 'height_cm',
    'Weight': 'weight_kg',
    '↓OVA': 'overall_rating',
    'POT': 'potential',
    'Value': 'value_eur',
    'Wage': 'wage_eur',
    'Release Clause': 'release_clause_eur'
})

# Dimension: Country
df_country = df[['nationality']].drop_duplicates()
df_country.to_csv('dim_country.csv', index=False)

# Dimension: Club
df_club = df[['club_name']].drop_duplicates()
df_club.to_csv('dim_club.csv', index=False)

# Dimension: Position
df_position = df[['player_position']].drop_duplicates()
df_position.to_csv('dim_position.csv', index=False)

# Dimension: Player
df_player = df[['player_name', 'age', 'height_cm', 'weight_kg']].drop_duplicates()
df_player.to_csv('dim_player.csv', index=False)


df_fact = df[['player_name', 'age', 'height_cm', 'weight_kg',
              'nationality', 'club_name', 'player_position',
              'overall_rating', 'potential', 'value_eur', 'wage_eur', 'release_clause_eur']]

df_fact.to_csv('fact_player_stats_raw.csv', index=False)

print("✅ All dimension and fact CSV files have been saved.")


✅ All dimension and fact CSV files have been saved.


In [68]:
import pandas as pd
from sqlalchemy import create_engine


user = 'root'
password = 'admin'  
host = 'localhost'
database = 'fifa_star_schema'


engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')


df_country = pd.read_csv('dim_country.csv')
df_club = pd.read_csv('dim_club.csv')
df_position = pd.read_csv('dim_position.csv')
df_player = pd.read_csv('dim_player.csv')
df_fact_raw = pd.read_csv('fact_player_stats_raw.csv')


df_country.to_sql('dim_country', con=engine, if_exists='append', index=False)
df_club.to_sql('dim_club', con=engine, if_exists='append', index=False)
df_position.to_sql('dim_position', con=engine, if_exists='append', index=False)
df_player.to_sql('dim_player', con=engine, if_exists='append', index=False)

print(" Dimension tables loaded.")





✅ Dimension tables loaded.


In [74]:
# Ensure consistent data types for 'height_cm' and 'weight_kg'
df_fact_raw['height_cm'] = df_fact_raw['height_cm'].astype('int64')
df_fact_raw['weight_kg'] = df_fact_raw['weight_kg'].astype('float64')
dim_player['height_cm'] = dim_player['height_cm'].astype('int64')
dim_player['weight_kg'] = dim_player['weight_kg'].astype('float64')

# Merge to get foreign keys
df_fact = df_fact_raw.merge(dim_country, on='nationality') \
                     .merge(dim_club, on='club_name') \
                     .merge(dim_position, on='player_position') \
                     .merge(dim_player, on=['player_name', 'age', 'height_cm', 'weight_kg'])

fact_df = df_fact[['player_id', 'club_id', 'country_id', 'position_id',
                   'overall_rating', 'potential', 'value_eur', 'wage_eur', 'release_clause_eur']]


fact_df.to_sql('fact_player_stats', con=engine, if_exists='append', index=False)

print("✅ Fact table loaded successfully.")


✅ Fact table loaded successfully.
